In [27]:
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as f
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from sklearn import preprocessing
from tqdm import tqdm
from sklearn.preprocessing import FunctionTransformer
import seaborn as sns 
from scipy import stats
import statsmodels.api as sm

from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor, BaggingRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import ElasticNet, Lasso
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import SVR
from sklearn.model_selection import KFold, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.neural_network import MLPRegressor
from scipy.stats import norm
import copy

In [2]:
train_df = pd.read_csv('./best_train.csv')
test_df = pd.read_csv('./best_test.csv')

In [3]:
features = list(train_df.columns)[:-1]
len(features)

159

In [7]:
def add_decomposition_features(train, test):
    from sklearn.decomposition import PCA, TruncatedSVD, FastICA
    from sklearn.random_projection import GaussianRandomProjection, SparseRandomProjection
    
    n_comp = 30
    
    # tSVD
    tsvd = TruncatedSVD(n_components=n_comp, random_state=420)
    tsvd_results_train = tsvd.fit_transform(train)
    tsvd_results_test = tsvd.transform(test)
    print("End tSVD")
    # PCA
    pca = PCA(n_components=n_comp, random_state=420)
    pca2_results_train = pca.fit_transform(train)
    pca2_results_test = pca.transform(test)
    print("End PCA")
    # ICA
    ica = FastICA(n_components=n_comp, random_state=420)
    ica2_results_train = ica.fit_transform(train)
    ica2_results_test = ica.transform(test)
    print("End ICA")
    # GRP
    grp = GaussianRandomProjection(n_components=n_comp, eps=0.1, random_state=420)
    grp_results_train = grp.fit_transform(train)
    grp_results_test = grp.transform(test)
    print("End GRP")
    # SRP
    srp = SparseRandomProjection(n_components=n_comp, dense_output=True, random_state=420)
    srp_results_train = srp.fit_transform(train)
    srp_results_test = srp.transform(test)
    print("End SRP")
    
    #NMF
    #nmf = NMF(n_components = n_comp, random_state=420)
    
    # Append decomposition components to datasets
    for i in tqdm(range(1, n_comp + 1)):
        train['pca_' + str(i)] = pca2_results_train[:, i - 1]
        test['pca_' + str(i)] = pca2_results_test[:, i - 1]
    
        train['ica_' + str(i)] = ica2_results_train[:, i - 1]
        test['ica_' + str(i)] = ica2_results_test[:, i - 1]
    
        train['tsvd_' + str(i)] = tsvd_results_train[:, i - 1]
        test['tsvd_' + str(i)] = tsvd_results_test[:, i - 1]
    
        train['grp_' + str(i)] = grp_results_train[:, i - 1]
        test['grp_' + str(i)] = grp_results_test[:, i - 1]
    
        train['srp_' + str(i)] = srp_results_train[:, i - 1]
        test['srp_' + str(i)] = srp_results_test[:, i - 1]
    
    print("add decomposition features done.")
    return train, test
    

In [8]:
X_train, X_test = add_decomposition_features(train_df[features], test_df[features])

End tSVD
End PCA
End ICA
End GRP
End SRP


 60%|███████████████████████████████████████████████████▌                                  | 18/30 [00:00<00:00, 30.50it/s]/data/hieunm/anaconda3/envs/action-env/lib/python3.7/site-packages/ipykernel_launcher.py:47: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
/data/hieunm/anaconda3/envs/action-env/lib/python3.7/site-packages/ipykernel_launcher.py:48: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
/data/hieunm/anaconda3/envs/action-env/lib/python3.7/site-packages/ipykernel_launcher.py:50: PerformanceWarning: DataFrame is highly fr

add decomposition features done.


In [16]:
from sklearn.model_selection import KFold
from catboost import CatBoostRegressor
import xgboost as xgb
from scipy.stats import gmean
target=["contest-tmp2m-14d__tmp2m"]
X = X_train.values
y = train_df[target[0]].values

skf = KFold(n_splits=5, random_state=2048, shuffle=True)
skf.get_n_splits(X, y)

5

In [17]:
train_index_list = []
test_index_list = []
train_x_list = []
val_x_list = []
train_y_list = []
val_y_list = []

for i, (train_index, test_index) in tqdm(enumerate(skf.split(X, y))):
    train_index_list.append(train_index)
    test_index_list.append(test_index)

    train_x_list.append(X[train_index, :])
    val_x_list.append(X[test_index, :])
    train_y_list.append(y[train_index])
    val_y_list.append(y[test_index])    

5it [00:54, 10.80s/it]


In [21]:
def train_xgboost(x, y):
    print(x.shape, y.shape)
    
    xgb_models = []
    for i in tqdm(range(1)):
        train_x, val_x = train_x_list[i], val_x_list[i]
        train_y, val_y = train_y_list[i], val_y_list[i]
        clf = CatBoostRegressor(iterations=4900, verbose=200)
        clf.fit(train_x, train_y, eval_set=[(val_x, val_y)], verbose=True, early_stopping_rounds=100)
        xgb_models.append(clf)
    
    return xgb_models
xgb_models = train_xgboost(X, y)

(375734, 309) (375734,)


  0%|                                                                                                | 0/1 [00:00<?, ?it/s]

Learning rate set to 0.047306
0:	learn: 9.4784843	test: 9.4576159	best: 9.4576159 (0)	total: 199ms	remaining: 16m 15s
1:	learn: 9.1016393	test: 9.0818338	best: 9.0818338 (1)	total: 368ms	remaining: 15m
2:	learn: 8.7455478	test: 8.7259172	best: 8.7259172 (2)	total: 462ms	remaining: 12m 33s
3:	learn: 8.4081012	test: 8.3897185	best: 8.3897185 (3)	total: 539ms	remaining: 11m
4:	learn: 8.0858643	test: 8.0682753	best: 8.0682753 (4)	total: 615ms	remaining: 10m 1s
5:	learn: 7.7792939	test: 7.7627119	best: 7.7627119 (5)	total: 687ms	remaining: 9m 20s
6:	learn: 7.4898570	test: 7.4741809	best: 7.4741809 (6)	total: 781ms	remaining: 9m 5s
7:	learn: 7.2104455	test: 7.1957601	best: 7.1957601 (7)	total: 853ms	remaining: 8m 41s
8:	learn: 6.9486106	test: 6.9351561	best: 6.9351561 (8)	total: 925ms	remaining: 8m 22s
9:	learn: 6.6972913	test: 6.6843833	best: 6.6843833 (9)	total: 1s	remaining: 8m 11s
10:	learn: 6.4613750	test: 6.4492719	best: 6.4492719 (10)	total: 1.08s	remaining: 8m 2s
11:	learn: 6.2329257

93:	learn: 1.8883037	test: 1.8865693	best: 1.8865693 (93)	total: 6.88s	remaining: 5m 51s
94:	learn: 1.8829549	test: 1.8812026	best: 1.8812026 (94)	total: 7.03s	remaining: 5m 55s
95:	learn: 1.8778715	test: 1.8759338	best: 1.8759338 (95)	total: 7.19s	remaining: 5m 59s
96:	learn: 1.8726687	test: 1.8706659	best: 1.8706659 (96)	total: 7.45s	remaining: 6m 9s
97:	learn: 1.8682668	test: 1.8662602	best: 1.8662602 (97)	total: 7.56s	remaining: 6m 10s
98:	learn: 1.8605391	test: 1.8586828	best: 1.8586828 (98)	total: 7.77s	remaining: 6m 16s
99:	learn: 1.8546252	test: 1.8527103	best: 1.8527103 (99)	total: 7.93s	remaining: 6m 20s
100:	learn: 1.8497264	test: 1.8479026	best: 1.8479026 (100)	total: 8.12s	remaining: 6m 25s
101:	learn: 1.8420713	test: 1.8403532	best: 1.8403532 (101)	total: 8.29s	remaining: 6m 29s
102:	learn: 1.8361493	test: 1.8344092	best: 1.8344092 (102)	total: 8.39s	remaining: 6m 30s
103:	learn: 1.8308871	test: 1.8290067	best: 1.8290067 (103)	total: 8.49s	remaining: 6m 31s
104:	learn: 1.

185:	learn: 1.5284016	test: 1.5275278	best: 1.5275278 (185)	total: 15s	remaining: 6m 19s
186:	learn: 1.5259897	test: 1.5250393	best: 1.5250393 (186)	total: 15s	remaining: 6m 19s
187:	learn: 1.5208321	test: 1.5198298	best: 1.5198298 (187)	total: 15.1s	remaining: 6m 18s
188:	learn: 1.5181937	test: 1.5173380	best: 1.5173380 (188)	total: 15.2s	remaining: 6m 18s
189:	learn: 1.5157348	test: 1.5148060	best: 1.5148060 (189)	total: 15.3s	remaining: 6m 18s
190:	learn: 1.5140927	test: 1.5131497	best: 1.5131497 (190)	total: 15.3s	remaining: 6m 18s
191:	learn: 1.5102266	test: 1.5091966	best: 1.5091966 (191)	total: 15.4s	remaining: 6m 17s
192:	learn: 1.5075395	test: 1.5065386	best: 1.5065386 (192)	total: 15.5s	remaining: 6m 17s
193:	learn: 1.5052802	test: 1.5042913	best: 1.5042913 (193)	total: 15.6s	remaining: 6m 17s
194:	learn: 1.5025547	test: 1.5015161	best: 1.5015161 (194)	total: 15.6s	remaining: 6m 17s
195:	learn: 1.5000105	test: 1.4990621	best: 1.4990621 (195)	total: 15.7s	remaining: 6m 16s
196

...

4604:	learn: 0.4048323	test: 0.4305796	best: 0.4305796 (4604)	total: 6m 31s	remaining: 25.1s
4605:	learn: 0.4048085	test: 0.4305594	best: 0.4305594 (4605)	total: 6m 31s	remaining: 25s
4606:	learn: 0.4047839	test: 0.4305335	best: 0.4305335 (4606)	total: 6m 31s	remaining: 24.9s
4607:	learn: 0.4047311	test: 0.4304937	best: 0.4304937 (4607)	total: 6m 31s	remaining: 24.8s
4608:	learn: 0.4047023	test: 0.4304644	best: 0.4304644 (4608)	total: 6m 31s	remaining: 24.7s
4609:	learn: 0.4046562	test: 0.4304256	best: 0.4304256 (4609)	total: 6m 31s	remaining: 24.6s
4610:	learn: 0.4046090	test: 0.4303802	best: 0.4303802 (4610)	total: 6m 31s	remaining: 24.5s
4611:	learn: 0.4045787	test: 0.4303525	best: 0.4303525 (4611)	total: 6m 31s	remaining: 24.5s
4612:	learn: 0.4045434	test: 0.4303178	best: 0.4303178 (4612)	total: 6m 31s	remaining: 24.4s
4613:	learn: 0.4044856	test: 0.4302527	best: 0.4302527 (4613)	total: 6m 31s	remaining: 24.3s
4614:	learn: 0.4044321	test: 0.4302007	best: 0.4302007 (4614)	total: 6m 

4694:	learn: 0.4010482	test: 0.4270787	best: 0.4270787 (4694)	total: 6m 38s	remaining: 17.4s
4695:	learn: 0.4010168	test: 0.4270505	best: 0.4270505 (4695)	total: 6m 38s	remaining: 17.3s
4696:	learn: 0.4009780	test: 0.4270136	best: 0.4270136 (4696)	total: 6m 38s	remaining: 17.2s
4697:	learn: 0.4009472	test: 0.4269858	best: 0.4269858 (4697)	total: 6m 38s	remaining: 17.1s
4698:	learn: 0.4009155	test: 0.4269561	best: 0.4269561 (4698)	total: 6m 38s	remaining: 17.1s
4699:	learn: 0.4008610	test: 0.4269050	best: 0.4269050 (4699)	total: 6m 38s	remaining: 17s
4700:	learn: 0.4007135	test: 0.4267668	best: 0.4267668 (4700)	total: 6m 38s	remaining: 16.9s
4701:	learn: 0.4006456	test: 0.4266924	best: 0.4266924 (4701)	total: 6m 38s	remaining: 16.8s
4702:	learn: 0.4006176	test: 0.4266655	best: 0.4266655 (4702)	total: 6m 38s	remaining: 16.7s
4703:	learn: 0.4005713	test: 0.4266142	best: 0.4266142 (4703)	total: 6m 38s	remaining: 16.6s
4704:	learn: 0.4005384	test: 0.4265862	best: 0.4265862 (4704)	total: 6m 

4785:	learn: 0.3972441	test: 0.4235418	best: 0.4235418 (4785)	total: 6m 45s	remaining: 9.65s
4786:	learn: 0.3972052	test: 0.4235041	best: 0.4235041 (4786)	total: 6m 45s	remaining: 9.56s
4787:	learn: 0.3971696	test: 0.4234755	best: 0.4234755 (4787)	total: 6m 45s	remaining: 9.48s
4788:	learn: 0.3971024	test: 0.4234068	best: 0.4234068 (4788)	total: 6m 45s	remaining: 9.4s
4789:	learn: 0.3970692	test: 0.4233740	best: 0.4233740 (4789)	total: 6m 45s	remaining: 9.31s
4790:	learn: 0.3970316	test: 0.4233400	best: 0.4233400 (4790)	total: 6m 45s	remaining: 9.23s
4791:	learn: 0.3969910	test: 0.4233059	best: 0.4233059 (4791)	total: 6m 45s	remaining: 9.14s
4792:	learn: 0.3969367	test: 0.4232556	best: 0.4232556 (4792)	total: 6m 45s	remaining: 9.06s
4793:	learn: 0.3969010	test: 0.4232240	best: 0.4232240 (4793)	total: 6m 45s	remaining: 8.97s
4794:	learn: 0.3968642	test: 0.4231884	best: 0.4231884 (4794)	total: 6m 45s	remaining: 8.88s
4795:	learn: 0.3968219	test: 0.4231504	best: 0.4231504 (4795)	total: 6m

4875:	learn: 0.3939145	test: 0.4204232	best: 0.4204232 (4875)	total: 6m 52s	remaining: 2.03s
4876:	learn: 0.3938849	test: 0.4203913	best: 0.4203913 (4876)	total: 6m 52s	remaining: 1.95s
4877:	learn: 0.3938536	test: 0.4203684	best: 0.4203684 (4877)	total: 6m 52s	remaining: 1.86s
4878:	learn: 0.3938184	test: 0.4203295	best: 0.4203295 (4878)	total: 6m 52s	remaining: 1.78s
4879:	learn: 0.3937736	test: 0.4202846	best: 0.4202846 (4879)	total: 6m 52s	remaining: 1.69s
4880:	learn: 0.3937431	test: 0.4202522	best: 0.4202522 (4880)	total: 6m 52s	remaining: 1.61s
4881:	learn: 0.3937051	test: 0.4202177	best: 0.4202177 (4881)	total: 6m 53s	remaining: 1.52s
4882:	learn: 0.3936571	test: 0.4201735	best: 0.4201735 (4882)	total: 6m 53s	remaining: 1.44s
4883:	learn: 0.3936209	test: 0.4201357	best: 0.4201357 (4883)	total: 6m 53s	remaining: 1.35s
4884:	learn: 0.3935637	test: 0.4200819	best: 0.4200819 (4884)	total: 6m 53s	remaining: 1.27s
4885:	learn: 0.3935112	test: 0.4200281	best: 0.4200281 (4885)	total: 6

100%|███████████████████████████████████████████████████████████████████████████████████████| 1/1 [07:49<00:00, 469.62s/it]


In [23]:
preds = []
for xgb_model in tqdm(xgb_models):
    preds.append(xgb_model.predict(X_test.values))
s = 0
for pred in preds:
    s += pred
s = s/1
df = pd.read_csv('./sample_solution.csv')
df['contest-tmp2m-14d__tmp2m'] = s
df.to_csv('submission_PCA.csv', index=False)

best_df = pd.read_csv('submission_5000_BFS_best.csv')
y_best = best_df[target[0]].values

from numpy import dot
from numpy.linalg import norm

cos_sim = dot(y_best, s)/(norm(y_best)*norm(s))
print("cos_sim with best submission:", cos_sim)

100%|████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.92s/it]


cos_sim with best submission: 0.9996729768799489


In [28]:
class StackingCVRegressorRetrained(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, regressors, meta_regressor, n_folds=2, use_features_in_secondary=False):
        self.regressors = regressors
        self.meta_regressor = meta_regressor
        self.n_folds = n_folds
        self.use_features_in_secondary = use_features_in_secondary
        self.regr_ = [clone(x) for x in self.regressors]
        self.meta_regr_ = clone(self.meta_regressor)

    def fit(self, train, y):
        if type(train) == pd.core.frame.DataFrame:            
            X = copy.deepcopy(train).values
            #ADD more features for meta regressor
            #train = add_features(train)
        elif type(train) == np.ndarray:            
            X = copy.deepcopy(train)

        kfold = KFold(n_splits=self.n_folds, shuffle=True)

        out_of_fold_predictions = np.zeros((X.shape[0], len(self.regressors)))

        # Create out-of-fold predictions for training meta-model
        for i, regr in enumerate(self.regr_):
            for train_idx, holdout_idx in kfold.split(X, y):
                instance = clone(regr)
                instance.fit(X[train_idx], y[train_idx])
                out_of_fold_predictions[holdout_idx, i] = instance.predict(X[holdout_idx])
       
        # Retrain base models on all data
        all_predictions = np.zeros((X.shape[0], len(self.regressors)))
        for i, regr in enumerate(self.regr_):
            regr.fit(X, y)
            all_predictions[:, i] = regr.predict(X)
        
        # Train meta-model
        #ADD more features
        if type(train) == pd.core.frame.DataFrame:
            train = add_features(train)
            X = train.values
        #X = train.values
        
        if self.use_features_in_secondary:
            self.meta_regr_.fit(np.hstack((X, out_of_fold_predictions)), y)
        else:
            self.meta_regr_.fit(out_of_fold_predictions, y)

        return self
    
    def predict(self, test):
        if type(test) == pd.core.frame.DataFrame:
            X = copy.deepcopy(test).values
        elif type(test) == np.ndarray:            
            X = copy.deepcopy(test)
        
        meta_features = np.column_stack([
            regr.predict(X) for regr in self.regr_
        ])
        
        if type(test) == pd.core.frame.DataFrame:
            #ADD more features
            test = add_features(test)        
            X = test.values

        if self.use_features_in_secondary:
            return self.meta_regr_.predict(np.hstack((X, meta_features)))
        else:
            return self.meta_regr_.predict(meta_features)

In [36]:
y_mean = train_df[target[0]].mean()
y_train = train_df[target[0]].values
print("y mean:", y_mean)

y mean: 11.858568220648424


In [34]:
en = make_pipeline(RobustScaler(), SelectFromModel(Lasso(alpha=0.03)), ElasticNet(alpha=0.001, l1_ratio=0.1))
    
adb = AdaBoostRegressor(DecisionTreeRegressor(max_depth=4), n_estimators=300, random_state=2017)

bag = BaggingRegressor(DecisionTreeRegressor(max_depth=4), n_estimators=300, random_state=2017)

rf = RandomForestRegressor(n_estimators=250, n_jobs=4, min_samples_split=25, min_samples_leaf=25)

et = ExtraTreesRegressor(n_estimators=250, n_jobs=4, min_samples_split=25, min_samples_leaf=25)

gbr = GradientBoostingRegressor(loss='huber', learning_rate=0.1, n_estimators=350, min_samples_split=25, min_samples_leaf=25)

xgbm = xgb.sklearn.XGBRegressor(max_depth=6, learning_rate=0.005, subsample=0.6, base_score=y_mean,
                            objective='reg:linear', n_estimators=1000)

nn = MLPRegressor(hidden_layer_sizes=(200, 400, 50), random_state =2017, early_stopping=True)

svm = SVR(kernel='poly', degree=3, gamma='auto', coef0=0.0, tol=0.001, C=1.0, epsilon=0.1, shrinking=True, cache_size=200, verbose=False, max_iter=-1)

In [37]:
gbr_results = cross_val_score(gbr, X_train.values, y_train, cv=5, scoring='r2')
print("gradient boosting regressor score: {:4f} ({:4f})".format(gbr_results.mean(), gbr_results.std()))

KeyboardInterrupt: 

In [ ]:
adb_results = cross_val_score(adb, train.values, y_train, cv=5, scoring='r2')
print("adboost regressor score: {:4f} ({:4f})".format(adb_results.mean(), results.std()))